In [1]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests


number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [3]:
#Initial table set

# Historical target table
Hist_table=HTable.read(1396, with_units=False)
Hist_table


Months,Entries
IntegerAndSexagesimal,Historical
31 ;,"01s 00 ; 33,18"
59 ;,"01s 28 ; 09,11"
90 ;,"02s 28 ; 42,19"
120 ;,"03s 28 ; 16,39"
151 ;,"04s 28 ; 49,57"
181 ;,"05s 28 ; 24,07"
212 ;,"06s 28 ; 57,25"
243 ;,"07s 29 ; 30,43"
273 ;,"08s 29 ; 04,53"


In [6]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(12):
    Test_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)
    Delta_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)


In [7]:
# mean motion parameter from the Parisian Alfonsine tables
daily_mean=Historical(Sexagesimal("0;59,8,19,37,19,13,56"),8)
REVO=Historical(Sexagesimal("6,0;0,0"),8)

for i in range(12):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=4):
        Test_recomp[i][1]=daily_mean*int(Hist_table[i][0])%REVO
        with set_precision(tmode=TruncatureMode.ROUND, pmode=2):
            Test_recomp[i][1]=1*Test_recomp[i][1]
            Delta_recomp[i][1]=3600*(Hist_table[i][1]-Test_recomp[i][1])

In [8]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(-0.8333333333333334, nan)

In [9]:
B.astype(float).style.background_gradient(axis=None)

,Entries
Months,
31 ;,0.000000
59 ;,0.000000
90 ;,-10.000000
120 ;,0.000000
151 ;,0.000000
181 ;,0.000000
212 ;,0.000000
243 ;,0.000000
273 ;,0.000000


In [10]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Solar mean months.csv')
B.to_csv('Solar mean months.zip', index=False, compression=compression_opts)